In [38]:
import time
import numpy as np
import pandas as pd
from typing import Dict, Union
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    NoSuchElementException,
    StaleElementReferenceException,
    TimeoutException
)
from webdriver_manager.chrome import ChromeDriverManager

# Set up Chrome driver
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument("--log-level=3")  # Suppress logs
options.add_argument("--disable-logging")  # Disable logging
options.add_argument("--silent")  # Silent mode
# options.add_argument("headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1920x1080")
options.add_argument("disable-gpu")
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=service)
driver.set_page_load_timeout(30)
link_all = [
    "https://batdongsan.com.vn/nha-dat-ban",
    "https://batdongsan.com.vn/nha-dat-cho-thue",
]
real_estate_type = [
    "nha-dat-ban",
    "nha-dat-cho-thue",
]

driver.get("https://batdongsan.com.vn/nha-dat-ban")

In [89]:
filter_button = driver.find_element(By.CSS_SELECTOR, "div[data-default-value='Loại nhà đất']")
filter_button.click()

# re_filter_box = driver.find_element(By.CSS_SELECTOR, ".mCSB_container")
re_filter_box = driver.find_element(By.CSS_SELECTOR, ".re__listing-search-select-dropdown.re__multiple.js__listing-search-select-select-dropdown.re__show-fade-in")
re_elems = re_filter_box.find_elements(By.CSS_SELECTOR, "li")

apply_button = re_filter_box.find_element(By.CSS_SELECTOR, ".re__btn.re__btn-pr-solid--sm.js__listing-search-select-apply-button")
all_types_elem = re_elems[0]

In [86]:
get_elems_actions = ActionChains(driver)
re_type_texts = set()
re_type_ids = set()
for li in re_elems[1:]:
    get_elems_actions.move_to_element(li).perform()
    try:
        ul = li.find_element(By.CSS_SELECTOR, "ul")
        li_elems = ul.find_elements(By.CSS_SELECTOR, "li")
        for li_elem in li_elems:
                re_type = li_elem.text
                re_type_texts.add(li_elem.text)
                re_type_ids.add(li_elem.get_attribute("value"))
    except NoSuchElementException:
            re_type_texts.add(li.text)
            re_type_ids.add(li.get_attribute("value"))

re_type_texts = list(re_type_texts)
re_type_ids = list(re_type_ids)

print(re_type_texts)
print(re_type_ids)
print(f"Real estate texts: {len(re_type_texts)}")
print(f"Real estate types: {len(re_type_ids)}")

['Condotel', 'Shophouse, nhà phố thương mại', 'Bán đất', 'Nhà riêng', 'Nhà mặt phố', 'Kho, nhà xưởng', 'Bất động sản khác', 'Nhà biệt thự, liền kề', 'Đất nền dự án', 'Chung cư mini, căn hộ dịch vụ']
['283', '562', '325', '41', '45', '40', '163', '575', '48', '650']
Real estate texts: 10
Real estate types: 10


In [92]:
check_elems_actions = ActionChains(driver) 
for id in re_type_ids:
    filter_button = driver.find_element(By.CSS_SELECTOR, "div[data-default-value='Loại nhà đất']")
    filter_button.click()
    re_filter_box = driver.find_element(By.CSS_SELECTOR, ".re__listing-search-select-dropdown.re__multiple.js__listing-search-select-select-dropdown.re__show-fade-in")
    re_elems = re_filter_box.find_elements(By.CSS_SELECTOR, "li")

    apply_button = re_filter_box.find_element(By.CSS_SELECTOR, ".re__btn.re__btn-pr-solid--sm.js__listing-search-select-apply-button")
    all_types_elem = re_elems[0]
    check_elems_actions.move_to_element(all_types_elem).perform()
    all_types_elem.click()
    time.sleep(1)
    all_types_elem.click()
    check_box = re_filter_box.find_element(By.CSS_SELECTOR, f"[value='{id}']")
    check_elems_actions.move_to_element(check_box).perform()
    check_box.click()
    print(f"Crawling {check_box.text}")
    apply_button.click()
    time.sleep(5)
    # Navigate back to the main page
    driver.get("https://batdongsan.com.vn/nha-dat-ban")

Crawling Bán đất
Crawling Condotel
Crawling Nhà biệt thự, liền kề
Crawling Nhà riêng
Crawling Kho, nhà xưởng
Crawling Đất nền dự án
Crawling Nhà mặt phố
Crawling Shophouse, nhà phố thương mại
Crawling Bất động sản khác
Crawling Chung cư mini, căn hộ dịch vụ
